In [51]:
import torch
from torch.autograd import Variable
from model import *
from dataset import *
import re

In [36]:
file = 'good models without Temperature/grimms_fairy_tails/best_checkpoint.pth.tar'
vocab_size = 87
seq_length = 30

print("=> loading checkpoint '{}'".format(file))
checkpoint = torch.load(file, map_location='cpu')

dataset = TextDataset(filename='rsc/book_EN_grimms_fairy_tails.txt', seq_length=30, batch_size=128,
                      train_steps=100)
model = TextGenerationModel(vocabulary_size=vocab_size, lstm_num_hidden=128,
                            lstm_num_layers=2, device='cpu',
                            dropout_prob=0.2)
model.eval()
optimizer = torch.optim.RMSprop(model.parameters())

best_accuracy = checkpoint['best_accuracy']
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
print("=> loaded checkpoint '{}' (epoch {}) with best accuracy: {:.4f}".format(file, checkpoint['step'], best_accuracy))

=> loading checkpoint 'good models without Temperature/grimms_fairy_tails/best_checkpoint.pth.tar'
Initialize dataset with 540241 characters, 87 unique.
=> loaded checkpoint 'good models without Temperature/grimms_fairy_tails/best_checkpoint.pth.tar' (epoch 382494) with best accuracy: 0.6958


In [37]:
# # Print sampled data
# with open('good models without Temperature/grimms_fairy_tails/generated_text.txt', 'r') as f:
#     content = f.readlines()
# content = [x.strip() for x in content[::2]]
# content[::25]

In [42]:
def sample_sequence(model, dataset, length=30):
    with torch.no_grad():
        sequence = Variable(torch.empty(1, 1)).long()
        sequence.random_(to=dataset.vocab_size)

        for t in range(0, seq_length):
            out_seq, _, _ = model(sequence)
            next_char = torch.tensor([[torch.argmax(out_seq, dim=2)[0, t]]]).long()
            sequence = torch.cat((sequence, next_char), dim=1)

        generated_sequence = dataset.convert_to_string(sequence.detach().cpu().numpy().squeeze())
    
    return generated_sequence
        

In [31]:
def predict_new_seq_given_seq(model, dataset, sentence="The sleeping beauty is", length=30):
    with torch.no_grad():
        in_seq = torch.tensor([[dataset._char_to_ix[sentence[-1]]]])
        sentence = sentence[:len(sentence)-1]
        sequence = torch.tensor([[dataset._char_to_ix[c] for c in sentence]])
        init_out, h, c = model(sequence)        
        hc_0 = tuple((h, c))
        
        for t in range(0, length):
            out, h, c = model(in_seq, hc_0)
            next_char = torch.tensor([[torch.argmax(out, dim=2)[0, t]]])
            in_seq = torch.cat((in_seq, next_char), dim=1)
            
        generated_seq = dataset.convert_to_string(in_seq.detach().numpy().squeeze())
        print(sentence + generated_seq)

In [90]:
predict_new_seq_given_seq(model, dataset, length=60)

The sleeping beauty is to be mayor.’ The man went out together, and the second son


In [89]:
predict_new_seq_given_seq(model, dataset, length=30)

The sleeping beauty is to be mayor.’ The man went ou
